I've explored Random Forests pretty thoroughly, I'm now going to experiment with using the xgboost implementation of gradient boosting. At this point in time, my initial data pre-processing isn't going to be particular sophisticated. I'll use pd.get_dummies to convert categorical variables into dummies and use random shuffling and forward filling to deal with NAs. Depending on how bloated the data preprocessing becomes, I may have to do this using AWS

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
#Importing data

t0 = time.time()

train = pd.read_csv('Data/train_transaction.csv')
test = pd.read_csv('Data/test_transaction.csv')

train_shape = train.shape
test_shape = test.shape

t1 = time.time()

print(train_shape, test_shape, t1-t0)

(590540, 394) (506691, 393) 126.07266283035278


In [3]:
#Creating function to deal with NAs by shuffling and forward filling.

def ffill(df):
    
    t0 = time.time()
    
    na_count = df.isna().sum().sum()
    while na_count>0:
        df = df.sample(frac=1)
        df = df.fillna(method='ffill',limit=10)
        na_count = df.isna().sum().sum()

    
    df = df.sort_index()
    t1 = time.time()

    return(df)
    print(t1-t0)

In [6]:
t0 = time.time()

#Storing dependent variable
fraud = train['isFraud']
train = train.drop('isFraud',axis =1)

#Stacking training and test design matrices to extract dummies
X = pd.concat([train,test], ignore_index=True)

del train, test

t1 = time.time()


print(t1-t0)

31.376667737960815


In [7]:
t0 = time.time()

strings = X.select_dtypes(include='object')
dummies = pd.get_dummies(strings)

del strings

t1 = time.time()

print(t1-t0)

30.266950130462646


In [8]:
t0 = time.time()

numerics = X.select_dtypes(exclude='object')
numerics = ffill(numerics)

t1 = time.time()

print(t1-t0, numerics.shape)

300.03569412231445 (1097231, 379)


In [9]:
X = pd.concat([numerics, dummies], axis=1)

print(X.shape)

(1097231, 531)


In [10]:
t0 = time.time()

X_train = X.iloc[:train_shape[0],:]
X_test = X.iloc[train_shape[0]:,:]

t1 = time.time()

print(t1-t0)

del X

32.70745396614075


I'm deleting a lot variables here because I need to be efficient in my RAM usage. I also use a crude timer to check where bottlenecks are.

In [11]:
train = pd.concat([fraud, X_train], axis=1)

train.to_csv('Data/train.csv', index=False)
X_test.to_csv('Data/test.csv', index=False)

NameError: name 'test' is not defined

In [12]:
X_test.to_csv('Data/test.csv', index=False)